# Database Management

## Notebook for creating and editing database tables/entries

In [2]:
# import sqlite3
# import numpy as np
# sqlite3.register_adapter(np.int64, lambda val: int(val))  # sqlite3 won't accept int > 8 bytes
# sqlite3.register_adapter(np.int32, lambda val: int(val))
import warnings
import mysql.connector

In [5]:
# db_dir = '/nfs/proj-repo/AAARG-dissertation'
# db_name = 'sumresults.db'
# db_path = db_dir + '/' + db_name

In [11]:
# def get_connection(path):
#     conn = sqlite3.connect(path)  # creates db if doesn't exist
#     c = conn.cursor()
#     return conn, c
def get_connection():
    db = mysql.connector.connect(
        host="172.30.64.94",
        user="root",
        password="aaargroot",
        database='aaargdb'
    )
    cursor = db.cursor(buffered=True)
    return db, cursor

In [18]:
TABLES = {}
TABLES['topic'] = """CREATE TABLE IF NOT EXISTS topics (
    id int NOT NULL,
    title varchar(1000),
    description varchar (1000),
    start int,
    end int,
    query text (1000),
    type varchar (1000),
    PRIMARY KEY (id)
)"""

TABLES['techniques'] = """ CREATE TABLE IF NOT EXISTS techniques (
    name varchar (500),
    description text,
    PRIMARY KEY (name)
)"""

TABLES['instances'] = """CREATE TABLE IF NOT EXISTS instances (
    instance int,
    technique varchar(500),
    temporal boolean,
    start_exec datetime,
    end_exec datetime,
    FOREIGN KEY (technique) REFERENCES techniques(name),
    PRIMARY KEY (instance)
)"""

TABLES['instance_meta'] = """CREATE TABLE IF NOT EXISTS instance_meta (
    topic_id int,
    instance int,
    summary longtext,
    streamids longtext,
    epoch_start int,
    epoch_end int,
    importance_score int,
    total_importance int,
    r1_precision FLOAT(53),
    r1_recall FLOAT(53),
    r1_fmeasure FLOAT(53),
    FOREIGN KEY (topic_id) REFERENCES topics(id),
    FOREIGN KEY (instance) REFERENCES instances(instance),
    PRIMARY KEY (topic_id, instance)
)"""

TABLES['nuggets'] = """CREATE TABLE IF NOT EXISTS nuggets (
    nugget_id varchar(200),
    topic_id int,
    importance int,
    nugget_text text,
    FOREIGN KEY (topic_id) REFERENCES topics(id),
    PRIMARY KEY (nugget_id, topic_id)
)"""

TABLES['nugget_instances'] = """CREATE TABLE IF NOT EXISTS nugget_instances (
    nugget_id varchar(200),
    topic_id int,
    instance int,
    technique varchar(500),
    is_complete_summary boolean,
    found boolean,
    FOREIGN KEY (nugget_id, topic_id) REFERENCES nuggets (nugget_id, topic_id),
    FOREIGN KEY (instance) REFERENCES instances (instance),
    FOREIGN KEY (technique) REFERENCES techniques (name)
)"""

In [19]:
def create_tables(conn, cursor):
    for name, table in TABLES.items():
        cursor.execute(table)
        print("Added " + str(name) + " table")
    conn.commit()
    print("Finshed adding tables")
    
conn, cursor = get_connection()
create_tables(cursor)
conn.close()

Added topic table
Added techniques table
Added instances table
Added instance_meta table
Added nuggets table
Added nugget_instances table
Finshed adding tables


## Pre-Populating Database

In [ ]:
def is_empty_table(cursor, table_name):
    exec_str = "SELECT * from " + table_name + " limit 1"
    cursor.execute(exec_str)
    count = cursor.rowcount
    print("is_empty_table count: " + str(count))
    if count > 0:
        return False
    elif count == 0:
        return True
    elif count < 0:
        warnings.warn("Using a non-buffered cursor, unable to effectively check rowcount")

In [ ]:
def populate_topics(conn, cursor, topics_df):
    if is_empty_table(cursor, "topics"):
        cols = ['id', 'title', 'description', 'start', 'end', 'query', 'type']
        entries = []
        for index, row in topics_df.iterrows():
            entry = []
            for col in cols:
                entry.append(row[col])
            entries.append(tuple(entry))
        print("entries len:" + str(len(entries)))
        cursor.executemany("""INSERT INTO topics (id, title, description, start, end, query, type) 
            VALUES(%s, %s, %s, %s, %s, %s, %s)""", entries)
        conn.commit()
        print("topics_df inserted into db")
    else:
        warnings.warn("The topics table already has entries")

In [ ]:
def populate_nuggets(conn, cursor, nugget_df):
    if is_empty_table(cursor, "nuggets"):
        # drop unneeded cols
        cols = ['nugget_id', 'topic_id', 'importance', 'nugget_text']
        df = nugget_df[cols]
        # drop repeats
        df = df.drop_duplicates(subset=['topic_id', 'nugget_id'])

        entries = []
        for index, row in df.iterrows():
            entry = []
            for col in cols:
                entry.append(row[col])
            entries.append(tuple(entry))
        cursor.executemany("""INSERT INTO nuggets (nugget_id, topic_id, importance, nugget_text)
            VALUES(%s, %s, %s, %s)""", entries)
        conn.commit()
        print("nuggets inserted into db")
    else:
        warnings.warn("The nuggets table already has entries")   

In [ ]:
# def populate_nuggets(conn, cursor, nugget_df):
#     cursor.execute('SELECT COUNT(nugget_id) FROM nuggets')
#     rowcount = cursor.fetchone()[0]
#     if rowcount == 0:
#         # drop unneeded cols
#         cols = ['nugget_id', 'topic_id', 'importance', 'nugget_text']
#         df = nugget_df[cols]
#         # drop repeats
#         df = df.drop_duplicates(subset=['topic_id', 'nugget_id'])

#         entries = []
#         for index, row in df.iterrows():
#             entry = []
#             for col in cols:
#                 entry.append(row[col])
#             entries.append(entry)
#         cursor.executemany('insert into nuggets values (?,?,?,?)', entries)
#     else:
#         warnings.warn("The nuggets table already has entries")
#     conn.commit()
#     print("populated nuggets")

## Insertions

In [ ]:
# def insert_